# Diving into Pinecone

In [2]:
# loading the API keys in environment variables
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [3]:
import pinecone

# initializing pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

In [5]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.1')

### Pinecone Indexes

In [15]:
pinecone.list_indexes()

['langchain-pinecone']

#### Creating an index

In [14]:
# creating an index
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ....')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
    print('Done')
else:
    print(f'Index {index_name} already exists!')

Creating index langchain-pinecone ....
Done


In [16]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

#### Deleting an index

In [13]:
# deleting an index
index_name = input('Enter Pinecone index to delete: ')
if index_name in pinecone.list_indexes():
    print(f'Deleting index {index_name} ... ')
    pinecone.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist!')

Enter Pinecone index to delete: react
Deleting index react ... 
Done


#### Getting index statistics

In [18]:
# getting statistics
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

#### Inserting into an index

In [19]:
# inserting some random vectors into a Pinecone index
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]

# vectors id
ids = list('abcde')

In [20]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

#### Updating a vector

In [21]:
# updating a vector
index.upsert(vectors=[('c', [0.3] * 1536)])

{'upserted_count': 1}

#### Fetching a vector

In [22]:
# fetching a vector
index = pinecone.Index('langchain-pinecone')
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

#### Deleting vectors

In [21]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [26]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [24]:
index.fetch(ids=['b'])

{'namespace': '', 'vectors': {}}

In [25]:
index.delete(delete_all=True)

{}

#### Querying

In [31]:
# querying 
queries = [[random.random() for _ in range(1536)] for v in range(2)]

In [32]:
index.query(
    queries=queries,
    top_k=3,
    include_values=False
)

{'results': [{'matches': [{'id': 'a', 'score': 0.767048299, 'values': []},
                          {'id': 'e', 'score': 0.763070583, 'values': []},
                          {'id': 'c', 'score': 0.757253408, 'values': []}],
              'namespace': ''},
             {'matches': [{'id': 'a', 'score': 0.762775779, 'values': []},
                          {'id': 'c', 'score': 0.75569278, 'values': []},
                          {'id': 'b', 'score': 0.748056, 'values': []}],
              'namespace': ''}]}